In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict

# Modelling Algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

# Configure visualisations
%matplotlib inline

In [ ]:
df = pd.read_csv("../datasets/winemag-data_first150k.csv", index_col=0)
df.head(10)

## Задача
Предсказать, что оценка вина будет выше, чем 90 баллов.

## 1. Предобработка 

### 1-1. Переведем колонку "points" в более подходящий для алгоритмов машинного обучения вид
1 - оценка выше, чем 90 баллов<br>
0 - оценка ниже, чем 90 баллов

In [ ]:
df["points"] = df["points"].apply(lambda points: 1 if points > 90 else 0)
df["points"].head(10)

### 1-2. Заменим nan на "Unknown" (в случае, если цена не указана, будем сбрасывать этот такие строки, так как непонятно, что делать с вином, у которого нет цены)

In [ ]:
df["price"] = df["price"].dropna()
df = df[pd.notnull(df["price"])]

df = df.replace(np.nan, 'Unknown', regex=True)
df.head(50)

### 1-3. Заменим все строковые колонки их категориальными эквивалентами

#### Присваиваем странам целочисленные коды

In [ ]:
df["country_codes"] = df["country"].astype("category").cat.codes
df["country_codes"].head()

In [ ]:
for country, code in zip(range(0, len(df["country"].astype("category").cat.categories)),
                         df["country"].astype("category").cat.categories):
    print("{0}:{1}".format(country, code))

In [ ]:
df.head()

#### designation

In [ ]:
Counter(df["designation"]).most_common(5)

Мы видим, что "designation" повторяются, значит их тоже можно перевести в в категории

In [ ]:
df["designation_codes"] = df["designation"].astype("category").cat.codes
df["designation_codes"].head()

#### province

In [ ]:
Counter(df["province"]).most_common(5)

Мы видим, что "province" повторяются, значит их тоже можно перевести в в категории

In [ ]:
df["province_codes"] = df["province"].astype("category").cat.codes
df["province_codes"].head()

#### region_1

In [ ]:
Counter(df["region_1"]).most_common(5)

Мы видим, что "region_1" повторяются, значит их тоже можно перевести в в категории

In [ ]:
df["region_1_codes"] = df["region_1"].astype("category").cat.codes
df["region_1_codes"].head()

#### region_2

In [ ]:
Counter(df["region_2"]).most_common(5)

Мы видим, что "region_2" повторяются, значит их тоже можно перевести в в категории

In [ ]:
df["region_2_codes"] = df["region_2"].astype("category").cat.codes
df["region_2_codes"].head()

#### variety

In [ ]:
Counter(df["variety"]).most_common(5)

Мы видим, что "province" повторяются, значит их тоже можно перевести в в категории

In [ ]:
df["variety_codes"] = df["variety"].astype("category").cat.codes
df["variety_codes"].head()

#### winery

In [ ]:
Counter(df["winery"]).most_common(5)

Мы видим, что "province" повторяются, значит их тоже можно перевести в в категории

In [ ]:
df["winery_codes"] = df["winery"].astype("category").cat.codes
df["winery_codes"].head()

In [ ]:
df_new = df[["price","country_codes",
                         "designation_codes","province_codes",
                         "region_1_codes","region_2_codes",
                         "variety_codes","winery_codes", "points"]]
df_new.head(10)

### 2. Разделим выборку на обучающую и тестовую, а также выделим целевую переменную из тестовой и тренировочной выборок
Строки из файла с номерами 2, 7, 12, ... отправляются в тестовую выборку.<br>
Остальные - в обучающую выборку.

In [ ]:
train_df = df_new[(df_new.index + 3) % 5 != 0]
train_df.head(10)

In [ ]:
test_df = df_new[(df_new.index + 3) % 5 == 0]
test_df.head(10)

In [ ]:
train_y = train_df["points"]
train_y.head(10)

In [ ]:
test_y = test_df["points"]
test_y.head(10)

In [ ]:
train_df = train_df.drop('points', 1)
test_df = test_df.drop('points', 1)

### 3. Обучение предсказательных моделей разными алгоритмами

#### Зададим random_state явно только для повторимости экспериментов

In [ ]:
my_rand_state = 241

### DecisionTreeClassifier

In [ ]:
decis_tree = DecisionTreeClassifier()
decis_tree.fit(train_df, train_y)

### KNeighborsClassifier

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=4) # количество соседей подобрано руками
neigh.fit(train_df, train_y)

### LogisticRegression

In [ ]:
log_regr = LogisticRegression()
log_regr.fit(train_df, train_y)

### GaussianNB

In [ ]:
gauss = GaussianNB()
gauss.fit(train_df, train_y)

### RandomForestClassifier

In [ ]:
random_forest = RandomForestClassifier()
random_forest.fit(train_df, train_y)

### GradientBoostingClassifier

In [ ]:
grad_boost = GradientBoostingClassifier()
grad_boost.fit(train_df, train_y)

### 4. Вероятность, с которой программа корректно отрабатывает на записях из тестовой выборки;

### DecisionTreeClassifier

In [ ]:
accuracy_score(test_y, decis_tree.predict(test_df))

### KNeighborsClassifier

In [ ]:
accuracy_score(test_y, neigh.predict(test_df))

### LogisticRegression

In [ ]:
accuracy_score(test_y, log_regr.predict(test_df))

### GaussianNB

In [ ]:
accuracy_score(test_y, gauss.predict(test_df))

### RandomForestClassifier

In [ ]:
accuracy_score(test_y, random_forest.predict(test_df))

### GradientBoostingClassifier

In [ ]:
accuracy_score(test_y, grad_boost.predict(test_df))

#### Лучшую точность на тестовых данных с гиперпараметрами по умолчанию показали "RandomForestClassifier" и "GradientBoostingClassifier"

### 5. Столбцы из выборки, которые наиболее влияют на корректное предсказание.

In [ ]:
def plot_model_var_imp( model , X , y, title ):
    imp = pd.DataFrame( 
        model.feature_importances_  , 
        columns = [ 'Importance' ] , 
        index = X.columns 
    )
    imp = imp.sort_values( [ 'Importance' ] , ascending = True )
    imp[ : 10 ].plot( kind = 'barh', title=title )

plot_model_var_imp( decis_tree , train_df , train_y, title="Desicion Tree" )
plot_model_var_imp( random_forest , train_df , train_y, title="Random forest" )
plot_model_var_imp( grad_boost , train_df , train_y, title="Gradient Boosting" )

#### На примерах трех алгоритмов мы можем увидеть, что наиболее важным критерием при оценке является "Цена" (столбец "price")